# Results

In [1]:
import pandas as pd

In [2]:
RESULTS_PATH = '../results/errors/'

In [3]:
def get_error_table(data_frame, LEVELS):
    errors = data_frame.groupby(by=['error_metric', 'level']).mean().reset_index().round(2)
    errors = pd.pivot(errors, values ='error', columns='error_metric', index='level')
    errors.index = LEVELS
    overall_error = data_frame.groupby(by='error_metric').mean()[['error']].transpose()
    overall_error.index = ['Overall']
    return errors.append(overall_error).round(2)

In [4]:
def get_percentage_improvement(base_df, approach_df):
    return (base_df - approach_df)/base_df*100

In [5]:
def get_results(data, model, LEVELS, FC_TYPES, column_order, short_horizon = False):
    column_names = {
        'base': 'BASE',
        'case1': 'ML_CASE1',
        'case1_mean': 'ML_CASE1_MEAN',
        'case1_lambda1': 'ML_CASE1_Lambda1',
        'case1_lambda1_mean': 'ML_CASE1_Lambda1_MEAN',
        'case1_lambda2': 'ML_CASE1_Lambda2',
        'case1_lambda2_mean': 'ML_CASE1_Lambda2_MEAN',
        'case1_one_seed': 'ML_CASE1_ONE_SEED',
        'case2': 'ML_CASE2',
        'case2_mean': 'ML_CASE2_MEAN',
        'case2_lambda1': 'ML_CASE2_Lambda1',
        'case2_lambda1_mean': 'ML_CASE2_Lambda1_MEAN',
        'case2_lambda2': 'ML_CASE2_Lambda2',
        'case2_lambda2_mean': 'ML_CASE2_Lambda2_MEAN',
        'case3': 'ML_CASE3',
        'case4': 'ML_CASE4',
        'bottomup': 'BU',
        'ols': 'OLS',
        'wls': 'WLS',
        'mintsample': 'MinT(Sample)',
        'mintshrink': 'MinT(Shrink)',
        'erm': 'ERM',
        'seeds_prev_params': 'PREV_PARAMS_SEEDS',
        'one_seed_prev_params': 'PREV_PARAMS_ONE_SEED',
        'seeds_prev_params_mean': 'PREV_PARAMS_SEEDS_MEAN'
    }
    
    error_dic = {}
    error_tables = {}
    error_percentages = {}
    final_percentage_tables = {}
    
    for fc_type in FC_TYPE:
        file_name = f'{data}_{model}'
        if short_horizon:
            file_name = f'{file_name}_short_horizon'
        error_dic[column_names[fc_type]] = pd.read_csv(f'{RESULTS_PATH}{file_name}_{fc_type}_errors.csv', index_col=0)
        
    for fc_name, error_df in error_dic.items():
        error_tables[fc_name] = get_error_table(error_df, LEVELS)
        
    for fc_name, error_df in error_tables.items():
        error_percentages[fc_name] = get_percentage_improvement(error_tables['BASE'], error_df)
        
    for error_metric in ['MSE', 'MAE', 'SMAPE']:
        dataframe_error = []
        for fc_name, error_df in error_percentages.items():
            if fc_name =='BASE':
                continue
            error_metric_percentages = error_df[[error_metric]]
            error_metric_percentages.columns = [fc_name]
            dataframe_error.append(error_metric_percentages)
        final_percentage_tables[error_metric] = pd.concat(dataframe_error, axis=1)[column_order]
        
    return error_tables['BASE'], final_percentage_tables

## New Results

## Prison

In [6]:
LEVELS_prison = ['Australia', 'State', 'Gender', 'Legal', 'Indigenous']
data_prison = 'prison'
FC_TYPE = ['base', 'bottomup', 'ols', 'wls', 'mintshrink', 'erm',
               'case1', 'case1_mean', 'case1_lambda1', 'case1_lambda1_mean', 'case1_lambda2', 'case1_lambda2_mean',
               'case2', 'case2_mean', 'case2_lambda1', 'case2_lambda1_mean', 'case2_lambda2', 'case2_lambda2_mean']
column_order_1 = ['BU', 'OLS', 'WLS', 'MinT(Shrink)', 'ERM', 'ML_CASE1', 'ML_CASE1_MEAN',
                 'ML_CASE1_Lambda1','ML_CASE1_Lambda1_MEAN','ML_CASE1_Lambda2','ML_CASE1_Lambda2_MEAN',
                  'ML_CASE2','ML_CASE2_MEAN','ML_CASE2_Lambda1','ML_CASE2_Lambda1_MEAN',
                  'ML_CASE2_Lambda2', 'ML_CASE2_Lambda2_MEAN']

model_arima ='arima'
base_errors_prison_arima, percentages_prison_arima = get_results(data_prison, model_arima, LEVELS_prison, FC_TYPE, column_order_1)



CASE 1: Only bottom level validation loss
CASE 2: Whole hierarchy validation loss

Lambda [0.0001, 0.1] 

Lambda1 -- Lambda =1

Lambda2 -- Lambda [1.1, 5]

In [17]:
percentages_prison_arima['MSE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Shrink),ERM,ML_CASE1,ML_CASE1_MEAN,ML_CASE1_Lambda1,ML_CASE1_Lambda1_MEAN,ML_CASE1_Lambda2,ML_CASE1_Lambda2_MEAN,ML_CASE2,ML_CASE2_MEAN,ML_CASE2_Lambda1,ML_CASE2_Lambda1_MEAN,ML_CASE2_Lambda2,ML_CASE2_Lambda2_MEAN
Australia,-1204.936229,-13.275846,-230.437789,-114.450766,-150.126572,-902.877381,-921.621992,-1187.037023,-1154.926767,-203.576973,-172.687790,-902.877381,-921.621992,-1080.427882,-1171.742917,-965.237615,-970.969927
State,-0.741349,8.906267,14.764202,20.707253,9.486221,6.118457,4.068817,-10.476289,-9.556132,47.350912,51.846227,6.118457,4.068817,1.830759,-5.693208,3.305897,2.336424
Gender,-4.889860,10.048874,13.384368,19.009192,15.612176,2.485159,0.415169,-12.069117,-11.132581,46.191271,50.895363,2.485159,0.415169,-1.179969,-7.740838,2.224576,2.061136
Legal,-12.232326,-10.070241,-11.037498,-3.632026,-59.544452,-7.105612,-8.635438,-18.380198,-16.119184,22.247171,19.688146,-7.105612,-8.635438,-8.213585,-13.391705,-8.302539,-8.382621
Indigenous,0.000000,1.381039,1.571899,8.681385,-36.429845,3.348354,2.298387,-3.928738,-2.451234,22.040824,20.750464,3.348354,2.298387,2.558476,-0.751334,2.695662,2.479319
Overall,-63.001288,3.695151,-4.110239,7.575853,-15.393930,-42.294162,-44.937381,-69.222360,-66.448091,26.688119,30.471385,-42.294162,-44.937381,-53.867637,-64.235912,-46.738537,-47.455759


In [19]:
percentages_prison_arima['MSE'].iloc[:, 5:].style.highlight_max(color = 'lightgreen', axis = 1)

,ML_CASE1,ML_CASE1_MEAN,ML_CASE1_Lambda1,ML_CASE1_Lambda1_MEAN,ML_CASE1_Lambda2,ML_CASE1_Lambda2_MEAN,ML_CASE2,ML_CASE2_MEAN,ML_CASE2_Lambda1,ML_CASE2_Lambda1_MEAN,ML_CASE2_Lambda2,ML_CASE2_Lambda2_MEAN
Australia,-902.877381,-921.621992,-1187.037023,-1154.926767,-203.576973,-172.687790,-902.877381,-921.621992,-1080.427882,-1171.742917,-965.237615,-970.969927
State,6.118457,4.068817,-10.476289,-9.556132,47.350912,51.846227,6.118457,4.068817,1.830759,-5.693208,3.305897,2.336424
Gender,2.485159,0.415169,-12.069117,-11.132581,46.191271,50.895363,2.485159,0.415169,-1.179969,-7.740838,2.224576,2.061136
Legal,-7.105612,-8.635438,-18.380198,-16.119184,22.247171,19.688146,-7.105612,-8.635438,-8.213585,-13.391705,-8.302539,-8.382621
Indigenous,3.348354,2.298387,-3.928738,-2.451234,22.040824,20.750464,3.348354,2.298387,2.558476,-0.751334,2.695662,2.479319
Overall,-42.294162,-44.937381,-69.222360,-66.448091,26.688119,30.471385,-42.294162,-44.937381,-53.867637,-64.235912,-46.738537,-47.455759


In [18]:
model_ets ='ets'
base_errors_prison_ets, percentages_prison_ets = get_results(data_prison, model_ets, LEVELS_prison, FC_TYPE, column_order_1)
percentages_prison_ets['MSE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Shrink),ERM,ML_CASE1,ML_CASE1_MEAN,ML_CASE1_Lambda1,ML_CASE1_Lambda1_MEAN,ML_CASE1_Lambda2,ML_CASE1_Lambda2_MEAN,ML_CASE2,ML_CASE2_MEAN,ML_CASE2_Lambda1,ML_CASE2_Lambda1_MEAN,ML_CASE2_Lambda2,ML_CASE2_Lambda2_MEAN
Australia,-3671.615741,-95.390013,-1439.641421,-932.717901,-5300.929077,-1958.030614,-1981.595850,-2866.015061,-2777.025390,-2695.406140,-2602.944786,-2621.183024,-2374.934330,-2510.402668,-2141.728770,-1995.221168,-1876.807859
State,23.569975,14.275249,20.715633,20.751485,-14.648431,25.886796,22.040608,28.262217,28.678703,30.734136,30.717010,31.086332,34.127619,31.363202,33.554561,31.707028,33.080479
Gender,12.276370,3.129698,7.906455,7.916996,-34.866193,14.878043,13.873077,17.463679,17.865566,20.315925,20.539995,20.648621,24.094080,20.306030,23.024502,21.897108,22.999930
Legal,6.081990,-0.466017,-4.214015,0.125546,-24.828064,6.036338,2.840160,7.897161,8.169145,9.665087,9.240427,9.598000,9.309613,9.479742,10.485179,8.563508,9.372716
Indigenous,0.000000,-5.287354,-14.794398,-7.243976,-86.333464,0.197037,-2.656325,1.481197,1.695622,2.874767,2.517540,2.690033,2.435687,2.664046,3.436501,2.089130,2.645780
Overall,-35.948105,4.501743,-11.767894,-3.108093,-102.595354,-11.035594,-14.127090,-21.195871,-19.633368,-16.604960,-15.421260,-15.401837,-9.996316,-13.907386,-6.999108,-6.589738,-3.909893


In [21]:
percentages_prison_ets['MSE'].iloc[:, 5:].style.highlight_max(color = 'lightgreen', axis = 1)

,ML_CASE1,ML_CASE1_MEAN,ML_CASE1_Lambda1,ML_CASE1_Lambda1_MEAN,ML_CASE1_Lambda2,ML_CASE1_Lambda2_MEAN,ML_CASE2,ML_CASE2_MEAN,ML_CASE2_Lambda1,ML_CASE2_Lambda1_MEAN,ML_CASE2_Lambda2,ML_CASE2_Lambda2_MEAN
Australia,-1958.030614,-1981.595850,-2866.015061,-2777.025390,-2695.406140,-2602.944786,-2621.183024,-2374.934330,-2510.402668,-2141.728770,-1995.221168,-1876.807859
State,25.886796,22.040608,28.262217,28.678703,30.734136,30.717010,31.086332,34.127619,31.363202,33.554561,31.707028,33.080479
Gender,14.878043,13.873077,17.463679,17.865566,20.315925,20.539995,20.648621,24.094080,20.306030,23.024502,21.897108,22.999930
Legal,6.036338,2.840160,7.897161,8.169145,9.665087,9.240427,9.598000,9.309613,9.479742,10.485179,8.563508,9.372716
Indigenous,0.197037,-2.656325,1.481197,1.695622,2.874767,2.517540,2.690033,2.435687,2.664046,3.436501,2.089130,2.645780
Overall,-11.035594,-14.127090,-21.195871,-19.633368,-16.604960,-15.421260,-15.401837,-9.996316,-13.907386,-6.999108,-6.589738,-3.909893


Short horizon

In [22]:
model_arima ='arima'
base_errors_prison_arima, percentages_prison_arima = get_results(data_prison, model_arima, LEVELS_prison, FC_TYPE, column_order_1, True)


In [23]:
percentages_prison_arima['MSE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Shrink),ERM,ML_CASE1,ML_CASE1_MEAN,ML_CASE1_Lambda1,ML_CASE1_Lambda1_MEAN,ML_CASE1_Lambda2,ML_CASE1_Lambda2_MEAN,ML_CASE2,ML_CASE2_MEAN,ML_CASE2_Lambda1,ML_CASE2_Lambda1_MEAN,ML_CASE2_Lambda2,ML_CASE2_Lambda2_MEAN
Australia,-3404.582840,-19.775671,-471.969377,-191.504184,-1519.071403,-1836.213430,-1919.672613,-3336.420326,-3138.158949,-1714.203104,-3942.554463,-1836.213430,-1919.672613,-2726.500329,-3228.850633,-2254.661185,-2275.492901
State,12.879618,6.734478,14.029956,17.439712,9.036993,17.408518,15.087357,-2.149563,-1.271665,41.724570,27.309656,17.408518,15.087357,13.692283,4.917550,12.616933,11.161647
Gender,13.392442,10.023429,14.739694,18.102436,16.442747,18.810520,16.596383,2.475224,3.517781,43.294603,28.176441,18.810520,16.596383,15.252492,8.191159,16.633475,16.112409
Legal,-5.374847,-14.083709,-11.962387,-7.717306,-63.458886,-0.870457,-2.983930,-17.800227,-14.823122,15.010035,-9.642062,-0.870457,-2.983930,-1.918467,-9.126515,-5.234346,-5.439736
Indigenous,0.000000,-5.975455,-5.263362,-0.729488,-54.737655,3.125744,1.636931,-8.619112,-6.481494,14.020959,-2.333504,3.125744,1.636931,2.325743,-2.616495,0.326744,-0.033986
Overall,-19.319347,2.554334,3.647937,9.568925,-20.821806,-1.978749,-4.790131,-31.170117,-28.135235,20.156545,-14.403940,-1.978749,-4.790131,-12.051874,-23.573353,-8.996404,-9.974963


In [24]:
percentages_prison_arima['MSE'].iloc[:, 5:].style.highlight_max(color = 'lightgreen', axis = 1)

,ML_CASE1,ML_CASE1_MEAN,ML_CASE1_Lambda1,ML_CASE1_Lambda1_MEAN,ML_CASE1_Lambda2,ML_CASE1_Lambda2_MEAN,ML_CASE2,ML_CASE2_MEAN,ML_CASE2_Lambda1,ML_CASE2_Lambda1_MEAN,ML_CASE2_Lambda2,ML_CASE2_Lambda2_MEAN
Australia,-1836.213430,-1919.672613,-3336.420326,-3138.158949,-1714.203104,-3942.554463,-1836.213430,-1919.672613,-2726.500329,-3228.850633,-2254.661185,-2275.492901
State,17.408518,15.087357,-2.149563,-1.271665,41.724570,27.309656,17.408518,15.087357,13.692283,4.917550,12.616933,11.161647
Gender,18.810520,16.596383,2.475224,3.517781,43.294603,28.176441,18.810520,16.596383,15.252492,8.191159,16.633475,16.112409
Legal,-0.870457,-2.983930,-17.800227,-14.823122,15.010035,-9.642062,-0.870457,-2.983930,-1.918467,-9.126515,-5.234346,-5.439736
Indigenous,3.125744,1.636931,-8.619112,-6.481494,14.020959,-2.333504,3.125744,1.636931,2.325743,-2.616495,0.326744,-0.033986
Overall,-1.978749,-4.790131,-31.170117,-28.135235,20.156545,-14.403940,-1.978749,-4.790131,-12.051874,-23.573353,-8.996404,-9.974963


In [25]:
model_ets ='ets'
base_errors_prison_ets, percentages_prison_ets = get_results(data_prison, model_ets, LEVELS_prison, FC_TYPE, column_order_1,
                                                            True)
percentages_prison_ets['MSE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Shrink),ERM,ML_CASE1,ML_CASE1_MEAN,ML_CASE1_Lambda1,ML_CASE1_Lambda1_MEAN,ML_CASE1_Lambda2,ML_CASE1_Lambda2_MEAN,ML_CASE2,ML_CASE2_MEAN,ML_CASE2_Lambda1,ML_CASE2_Lambda1_MEAN,ML_CASE2_Lambda2,ML_CASE2_Lambda2_MEAN
Australia,-3033.353567,-15.322609,-1002.874308,-565.991246,-2921.571633,-641.821581,-639.992423,-1741.339062,-1626.389271,-1531.972284,-1392.616576,-1427.920894,-1079.833761,-1253.757484,-835.870255,-705.844348,-625.051447
State,25.769501,10.728614,18.853948,17.545933,15.893155,19.603229,13.689340,29.315392,29.521898,31.403147,30.709172,32.430924,34.462417,32.294343,33.385164,29.124148,30.425919
Gender,17.187082,3.108864,8.698361,7.558099,8.613978,11.124900,8.321485,21.014729,21.238537,23.285472,22.769527,24.339031,26.000879,23.915694,25.314284,21.474946,22.625923
Legal,6.290859,-7.167135,-6.374773,-4.674812,-6.987274,-1.883968,-10.531003,6.920080,7.132614,8.513593,7.146450,9.016492,6.163635,8.568799,8.719007,3.575771,4.606311
Indigenous,0.000000,-10.983168,-17.938420,-12.922318,-39.317612,-6.945100,-14.722589,0.526532,0.699836,1.811305,0.670265,1.837639,-0.860635,1.735540,1.805273,-2.074350,-1.388049
Overall,-6.138350,2.815010,-0.256886,3.008417,-18.444188,5.472439,-0.131030,6.429279,7.508467,9.971181,10.226937,11.589174,14.776460,12.638035,16.707593,13.477374,15.223780


In [26]:
percentages_prison_ets['MSE'].iloc[:, 5:].style.highlight_max(color = 'lightgreen', axis = 1)

,ML_CASE1,ML_CASE1_MEAN,ML_CASE1_Lambda1,ML_CASE1_Lambda1_MEAN,ML_CASE1_Lambda2,ML_CASE1_Lambda2_MEAN,ML_CASE2,ML_CASE2_MEAN,ML_CASE2_Lambda1,ML_CASE2_Lambda1_MEAN,ML_CASE2_Lambda2,ML_CASE2_Lambda2_MEAN
Australia,-641.821581,-639.992423,-1741.339062,-1626.389271,-1531.972284,-1392.616576,-1427.920894,-1079.833761,-1253.757484,-835.870255,-705.844348,-625.051447
State,19.603229,13.689340,29.315392,29.521898,31.403147,30.709172,32.430924,34.462417,32.294343,33.385164,29.124148,30.425919
Gender,11.124900,8.321485,21.014729,21.238537,23.285472,22.769527,24.339031,26.000879,23.915694,25.314284,21.474946,22.625923
Legal,-1.883968,-10.531003,6.920080,7.132614,8.513593,7.146450,9.016492,6.163635,8.568799,8.719007,3.575771,4.606311
Indigenous,-6.945100,-14.722589,0.526532,0.699836,1.811305,0.670265,1.837639,-0.860635,1.735540,1.805273,-2.074350,-1.388049
Overall,5.472439,-0.131030,6.429279,7.508467,9.971181,10.226937,11.589174,14.776460,12.638035,16.707593,13.477374,15.223780


## Tourism

In [27]:
LEVELS_tourism = ['Australia', 'States', 'Regions']
data_tourism = 'tourism'

FC_TYPE = ['base', 'bottomup', 'ols', 'wls', 'mintsample', 'mintshrink', 'erm',
               'case1', 'case1_mean', 'case1_lambda1', 'case1_lambda1_mean', 'case1_lambda2', 'case1_lambda2_mean',
               'case2', 'case2_mean', 'case2_lambda1', 'case2_lambda1_mean', 'case2_lambda2', 'case2_lambda2_mean']
column_order_1 = ['BU', 'OLS', 'WLS', 'MinT(Sample)','MinT(Shrink)', 'ERM', 'ML_CASE1', 'ML_CASE1_MEAN',
                 'ML_CASE1_Lambda1','ML_CASE1_Lambda1_MEAN','ML_CASE1_Lambda2','ML_CASE1_Lambda2_MEAN',
                  'ML_CASE2','ML_CASE2_MEAN','ML_CASE2_Lambda1','ML_CASE2_Lambda1_MEAN',
                  'ML_CASE2_Lambda2', 'ML_CASE2_Lambda2_MEAN',]


model_arima ='arima'
base_errors_tourism_arima, percentages_tourism_arima = get_results(data_tourism, model_arima, LEVELS_tourism, FC_TYPE, column_order_1)

In [28]:
percentages_tourism_arima ['MSE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Sample),MinT(Shrink),ERM,ML_CASE1,ML_CASE1_MEAN,ML_CASE1_Lambda1,ML_CASE1_Lambda1_MEAN,ML_CASE1_Lambda2,ML_CASE1_Lambda2_MEAN,ML_CASE2,ML_CASE2_MEAN,ML_CASE2_Lambda1,ML_CASE2_Lambda1_MEAN,ML_CASE2_Lambda2,ML_CASE2_Lambda2_MEAN
Australia,-336.612973,-15.402187,-186.242951,-32.443611,-102.650850,-0.952876,-102.773745,-100.625579,-2.588107,9.933037,-145.526787,-143.375279,-35.991282,-35.882361,-146.752803,-145.250560,-66.593739,-71.240518
States,-69.790932,28.159025,-13.739921,35.504637,12.530673,42.812455,14.346513,14.649768,37.068438,44.279364,-5.844932,-5.333560,28.957981,28.615112,-11.822863,-11.772848,25.943950,24.399927
Regions,0.000000,16.233769,17.287729,23.978228,20.335997,14.419214,14.572254,14.605292,18.613858,19.651973,10.721102,10.846287,17.002606,16.892998,9.323549,9.507608,15.627896,15.283159
Overall,-198.046179,3.605229,-98.842832,-1.698437,-46.294762,15.599435,-46.664407,-45.441436,13.537692,22.559528,-76.082462,-74.776077,-6.748451,-6.819672,-78.873542,-78.045812,-23.888113,-26.863459


In [30]:
percentages_tourism_arima ['MSE'].iloc[:, 6:].style.highlight_max(color = 'lightgreen', axis = 1)

,ML_CASE1,ML_CASE1_MEAN,ML_CASE1_Lambda1,ML_CASE1_Lambda1_MEAN,ML_CASE1_Lambda2,ML_CASE1_Lambda2_MEAN,ML_CASE2,ML_CASE2_MEAN,ML_CASE2_Lambda1,ML_CASE2_Lambda1_MEAN,ML_CASE2_Lambda2,ML_CASE2_Lambda2_MEAN
Australia,-102.773745,-100.625579,-2.588107,9.933037,-145.526787,-143.375279,-35.991282,-35.882361,-146.752803,-145.250560,-66.593739,-71.240518
States,14.346513,14.649768,37.068438,44.279364,-5.844932,-5.333560,28.957981,28.615112,-11.822863,-11.772848,25.943950,24.399927
Regions,14.572254,14.605292,18.613858,19.651973,10.721102,10.846287,17.002606,16.892998,9.323549,9.507608,15.627896,15.283159
Overall,-46.664407,-45.441436,13.537692,22.559528,-76.082462,-74.776077,-6.748451,-6.819672,-78.873542,-78.045812,-23.888113,-26.863459


In [32]:
model_ets ='ets'
base_errors_tourism_ets, percentages_tourism_ets = get_results(data_tourism, model_ets, LEVELS_tourism, FC_TYPE, column_order_1)

In [33]:
percentages_tourism_ets['MSE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Sample),MinT(Shrink),ERM,ML_CASE1,ML_CASE1_MEAN,ML_CASE1_Lambda1,ML_CASE1_Lambda1_MEAN,ML_CASE1_Lambda2,ML_CASE1_Lambda2_MEAN,ML_CASE2,ML_CASE2_MEAN,ML_CASE2_Lambda1,ML_CASE2_Lambda1_MEAN,ML_CASE2_Lambda2,ML_CASE2_Lambda2_MEAN
Australia,-74.350078,0.996985,-32.792912,26.454736,-23.578552,34.256214,-18.335489,-2.190778,-4.950822,-20.597527,53.852066,54.328059,53.659942,51.467112,14.134104,36.120810,54.117239,52.622808
States,-37.322787,2.967490,-8.087188,31.235177,-2.283716,-36.878294,0.220360,10.019350,0.350575,-8.348524,41.213899,41.980650,42.963272,40.862065,17.746094,32.378966,42.727274,41.504603
Regions,0.000000,6.104559,10.261384,7.754256,10.001538,-111.025210,7.162891,8.995216,6.332391,4.867548,13.697656,13.811089,14.894083,14.428712,9.489702,12.094508,13.985230,13.737063
Overall,-58.869128,1.931734,-23.095100,25.726605,-15.559769,4.279624,-11.710646,1.629641,-2.672845,-15.392506,47.144240,47.650606,47.522001,45.516199,14.497903,32.965267,47.691937,46.378631


In [34]:
percentages_tourism_ets['MSE'].iloc[:, 6:].style.highlight_max(color = 'lightgreen', axis = 1)

,ML_CASE1,ML_CASE1_MEAN,ML_CASE1_Lambda1,ML_CASE1_Lambda1_MEAN,ML_CASE1_Lambda2,ML_CASE1_Lambda2_MEAN,ML_CASE2,ML_CASE2_MEAN,ML_CASE2_Lambda1,ML_CASE2_Lambda1_MEAN,ML_CASE2_Lambda2,ML_CASE2_Lambda2_MEAN
Australia,-18.335489,-2.190778,-4.950822,-20.597527,53.852066,54.328059,53.659942,51.467112,14.134104,36.120810,54.117239,52.622808
States,0.220360,10.019350,0.350575,-8.348524,41.213899,41.980650,42.963272,40.862065,17.746094,32.378966,42.727274,41.504603
Regions,7.162891,8.995216,6.332391,4.867548,13.697656,13.811089,14.894083,14.428712,9.489702,12.094508,13.985230,13.737063
Overall,-11.710646,1.629641,-2.672845,-15.392506,47.144240,47.650606,47.522001,45.516199,14.497903,32.965267,47.691937,46.378631


Short Horizon

In [35]:
base_errors_tourism_arima, percentages_tourism_arima = get_results(data_tourism, model_arima, LEVELS_tourism, FC_TYPE, 
                                                                   column_order_1, True)

In [36]:
percentages_tourism_arima ['MSE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Sample),MinT(Shrink),ERM,ML_CASE1,ML_CASE1_MEAN,ML_CASE1_Lambda1,ML_CASE1_Lambda1_MEAN,ML_CASE1_Lambda2,ML_CASE1_Lambda2_MEAN,ML_CASE2,ML_CASE2_MEAN,ML_CASE2_Lambda1,ML_CASE2_Lambda1_MEAN,ML_CASE2_Lambda2,ML_CASE2_Lambda2_MEAN
Australia,-270.441142,-14.214972,-156.302629,-21.756588,-86.739568,-2.155216,-77.919881,-72.419089,1.581177,11.818582,-119.559186,-116.296745,-14.959913,-20.537647,-116.690844,-114.601786,-42.955614,-45.076463
States,-54.708017,29.896808,-7.138181,40.626618,17.024754,44.990441,23.337688,23.890463,41.929774,48.345707,0.770110,1.842440,38.932495,36.737117,-0.912410,-0.740975,34.409654,33.854381
Regions,0.000000,12.421302,14.706956,21.072981,16.879456,13.058881,12.530202,12.603748,15.590601,16.477243,8.657997,8.873477,15.121030,14.708536,8.088807,8.264066,13.372659,13.162238
Overall,-165.095281,3.085456,-85.827512,3.473199,-39.983989,14.374362,-33.871639,-30.672318,15.873930,23.570646,-64.147730,-61.999347,5.812341,2.023119,-63.166218,-61.941064,-11.191335,-12.555407


In [37]:
model_ets ='ets'
base_errors_tourism_ets, percentages_tourism_ets = get_results(data_tourism, model_ets, LEVELS_tourism, FC_TYPE, column_order_1,
                                                              True)
percentages_tourism_ets['MSE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Sample),MinT(Shrink),ERM,ML_CASE1,ML_CASE1_MEAN,ML_CASE1_Lambda1,ML_CASE1_Lambda1_MEAN,ML_CASE1_Lambda2,ML_CASE1_Lambda2_MEAN,ML_CASE2,ML_CASE2_MEAN,ML_CASE2_Lambda1,ML_CASE2_Lambda1_MEAN,ML_CASE2_Lambda2,ML_CASE2_Lambda2_MEAN
Australia,-75.219439,2.487777,-29.676360,7.617090,-20.707002,78.937392,-9.462032,0.476217,-3.766515,-20.941174,51.523673,51.365423,49.407154,47.613899,7.513000,35.669695,52.196494,50.077909
States,-5.809304,7.375898,16.004130,42.280236,21.730145,27.476677,30.805816,35.913708,27.987033,19.662224,60.154336,60.003404,59.693884,58.383952,37.951200,52.979971,61.142022,59.717119
Regions,0.000000,1.256380,9.847346,-0.727899,8.877108,-85.805523,7.537024,8.515361,5.922127,4.503460,12.183010,12.005577,12.910993,12.562378,7.881863,10.713450,12.164591,11.966809
Overall,-50.104556,3.602071,-14.147982,15.557691,-6.996159,49.394179,2.438070,10.255136,5.244934,-8.109573,49.765769,49.607450,48.353697,46.827627,15.257227,37.551364,50.449029,48.698621


## Wikipedia

In [38]:
LEVELS_wiki = ['Total', 'Language', 'Access', 'Agent', 'Article']

data_wiki = 'wikipedia'
FC_TYPE = ['base', 'bottomup', 'ols', 'wls', 'mintshrink', 'erm',
               'case1', 'case1_mean', 'case1_lambda1', 'case1_lambda1_mean', 'case1_lambda2', 'case1_lambda2_mean',
               'case2', 'case2_mean', 'case2_lambda1', 'case2_lambda1_mean', 'case2_lambda2', 'case2_lambda2_mean']
column_order_1 = ['BU', 'OLS', 'WLS', 'MinT(Shrink)', 'ERM', 'ML_CASE1', 'ML_CASE1_MEAN',
                 'ML_CASE1_Lambda1','ML_CASE1_Lambda1_MEAN','ML_CASE1_Lambda2','ML_CASE1_Lambda2_MEAN',
                  'ML_CASE2','ML_CASE2_MEAN','ML_CASE2_Lambda1','ML_CASE2_Lambda1_MEAN',
                  'ML_CASE2_Lambda2', 'ML_CASE2_Lambda2_MEAN']

model_arima ='arima'
base_errors_wiki_arima, percentages_wiki_arima = get_results(data_wiki, model_arima, LEVELS_wiki, FC_TYPE, column_order_1)



In [39]:
percentages_wiki_arima ['MSE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Shrink),ERM,ML_CASE1,ML_CASE1_MEAN,ML_CASE1_Lambda1,ML_CASE1_Lambda1_MEAN,ML_CASE1_Lambda2,ML_CASE1_Lambda2_MEAN,ML_CASE2,ML_CASE2_MEAN,ML_CASE2_Lambda1,ML_CASE2_Lambda1_MEAN,ML_CASE2_Lambda2,ML_CASE2_Lambda2_MEAN
Total,30.844347,5.181596,55.856905,-1015.245576,-58.808207,49.211134,48.941658,43.753214,48.020864,59.601223,59.502469,69.655092,69.561335,45.011288,49.326788,51.241818,56.218813
Language,25.433718,-4.844533,37.651617,-787.849116,-86.594101,39.837009,39.606787,36.625417,40.418944,47.339274,47.369780,60.004908,60.135691,37.989515,41.839146,41.669590,45.809496
Access,32.080825,-1.230499,-129.978518,-606.638972,-93.439680,42.724063,42.548695,40.345297,43.168231,46.914236,46.910472,58.344131,58.382167,41.369591,44.225420,44.452043,47.422728
Agent,-896.607505,1.062973,-155.517605,-870.377117,-467.556773,-4491.012983,-4494.224752,-4496.642105,-4456.472731,-4443.350293,-4443.955338,-4017.260697,-4012.599400,-4474.997103,-4427.626112,-4432.211651,-4368.476816
Article,0.000000,-23.865074,-633.396724,-523.974750,-545.716574,-1490.464638,-1490.473962,-1490.504926,-1490.477909,-1490.450610,-1490.454954,-1490.385049,-1490.348108,-1490.489342,-1490.467567,-1490.464095,-1490.440613
Overall,-109.520179,-1.531834,-73.321581,-827.643424,-175.942592,-763.386431,-764.040108,-767.331960,-758.521637,-750.170135,-750.288324,-678.978224,-678.285297,-763.201697,-753.296424,-753.286906,-740.648689


In [40]:
model_ets ='ets'
base_errors_wiki_ets, percentages_wiki_ets = get_results(data_wiki, model_ets, LEVELS_wiki, FC_TYPE, column_order_1)
percentages_wiki_ets ['MSE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Shrink),ERM,ML_CASE1,ML_CASE1_MEAN,ML_CASE1_Lambda1,ML_CASE1_Lambda1_MEAN,ML_CASE1_Lambda2,ML_CASE1_Lambda2_MEAN,ML_CASE2,ML_CASE2_MEAN,ML_CASE2_Lambda1,ML_CASE2_Lambda1_MEAN,ML_CASE2_Lambda2,ML_CASE2_Lambda2_MEAN
Total,25.519796,5.902553,45.529253,-918.177460,-195.725711,36.030493,36.222535,26.557528,27.786007,28.717194,28.812484,44.701603,45.078878,52.585191,49.908474,51.316912,52.128466
Language,24.042062,4.836913,34.923472,-760.945135,-172.020681,30.319137,30.449109,24.674203,25.422918,26.066765,26.112637,36.076646,36.287792,39.663192,37.609829,40.273182,40.825614
Access,-19.433570,-16.747319,-204.655993,-773.003755,-2265.617677,-14.105532,-13.983647,-18.898840,-18.278909,-17.439580,-17.385724,-8.667327,-8.503084,-1.083176,-2.483462,-4.924566,-4.277183
Agent,-944.111416,-0.320377,-108.777567,-548.252168,-1328.311923,-4490.745352,-4489.362125,-4568.463024,-4559.574144,-4549.178649,-4548.060118,-4411.041402,-4407.607438,-3861.082993,-3844.542966,-4305.942031,-4297.456445
Article,0.000000,-26.225274,-825.784926,-530.736817,-1485.045776,-1635.668146,-1635.658758,-1635.713835,-1635.708495,-1635.708506,-1635.707044,-1635.623937,-1635.613803,-1635.563819,-1635.577502,-1635.583447,-1635.580433
Overall,-136.236013,-1.058154,-90.868802,-769.777012,-745.227696,-837.779878,-837.439045,-855.723283,-853.580996,-851.314199,-851.082164,-819.728797,-818.966376,-727.794215,-726.884091,-799.043448,-797.171133


Short Horizon

In [41]:
model_arima ='arima'
base_errors_wiki_arima, percentages_wiki_arima = get_results(data_wiki, model_arima, LEVELS_wiki, FC_TYPE, 
                                                             column_order_1, True)

percentages_wiki_arima ['MSE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Shrink),ERM,ML_CASE1,ML_CASE1_MEAN,ML_CASE1_Lambda1,ML_CASE1_Lambda1_MEAN,ML_CASE1_Lambda2,ML_CASE1_Lambda2_MEAN,ML_CASE2,ML_CASE2_MEAN,ML_CASE2_Lambda1,ML_CASE2_Lambda1_MEAN,ML_CASE2_Lambda2,ML_CASE2_Lambda2_MEAN
Total,34.430066,5.866183,84.643106,-306.015140,36.706810,53.518206,53.245146,47.750191,53.109889,66.587319,65.374864,80.913462,80.617839,47.742663,52.861890,55.120517,60.980945
Language,31.022679,-5.687649,57.229309,-236.753104,38.952864,47.566566,47.308682,44.105587,49.391283,58.313438,57.213183,76.421166,76.416943,44.098052,49.173359,49.145752,54.369935
Access,39.188540,-2.158041,-61.066888,-207.485717,20.910009,52.758443,52.534104,50.033060,54.393686,59.735477,58.875108,78.419882,78.451370,50.033126,54.225571,54.566869,58.769901
Agent,-448.207170,0.726583,-28.733745,-356.574686,-194.545356,-2925.715697,-2928.106099,-2950.699208,-2904.538153,-2844.101473,-2853.674795,-2570.547485,-2570.592632,-2950.699531,-2906.441772,-2904.576774,-2856.477519
Article,0.000000,-90.694764,-2214.064776,-1712.477008,-1270.696617,-5142.804763,-5142.828468,-5143.013502,-5142.782400,-5142.546192,-5142.591085,-5141.661290,-5141.539655,-5143.013260,-5142.791979,-5142.738513,-5142.548502
Overall,-33.401502,-0.687784,0.689528,-302.958070,-19.043749,-448.651854,-449.203711,-455.926408,-445.153009,-427.820606,-430.092229,-375.995233,-376.122448,-455.931604,-445.606998,-444.330648,-433.114205


In [42]:
model_ets ='ets'
base_errors_wiki_ets, percentages_wiki_ets = get_results(data_wiki, model_ets, LEVELS_wiki, FC_TYPE, 
                                                         column_order_1, True)
percentages_wiki_ets ['MSE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Shrink),ERM,ML_CASE1,ML_CASE1_MEAN,ML_CASE1_Lambda1,ML_CASE1_Lambda1_MEAN,ML_CASE1_Lambda2,ML_CASE1_Lambda2_MEAN,ML_CASE2,ML_CASE2_MEAN,ML_CASE2_Lambda1,ML_CASE2_Lambda1_MEAN,ML_CASE2_Lambda2,ML_CASE2_Lambda2_MEAN
Total,26.404619,5.910642,85.715717,-251.133556,-58.930852,38.227349,38.443728,27.496807,28.823778,29.823569,29.929454,48.747595,49.267094,85.638664,86.377985,60.311464,61.454495
Language,29.741725,6.380009,70.537365,-203.953709,-41.085939,38.606496,38.775417,30.581556,31.584764,32.481915,32.560347,47.038108,47.391693,84.674981,85.135299,56.327722,57.194937
Access,-19.452671,-28.554225,-126.507773,-311.965637,-1395.161529,-10.131545,-9.932127,-18.561395,-17.518132,-16.277485,-16.192674,-0.683486,-0.337118,42.688090,43.241407,9.000763,10.073109
Agent,-537.007796,-1.972247,-11.825264,-198.868422,-716.532582,-3342.922595,-3341.472487,-3400.234771,-3393.166533,-3385.706689,-3385.199820,-3275.106781,-3272.439501,-2829.162338,-2822.999335,-3198.290779,-3190.071148
Article,0.000000,-105.110147,-3038.651520,-1865.331126,-6679.504617,-6016.422864,-6016.388999,-6016.790941,-6016.747341,-6016.726390,-6016.712549,-6016.040282,-6015.994893,-6014.372354,-6014.342308,-6015.614961,-6015.581648
Overall,-56.723104,-0.508650,-3.705108,-263.602996,-393.482510,-534.154927,-533.783216,-550.282528,-548.287974,-546.402316,-546.251225,-516.447822,-515.697202,-422.116635,-420.723042,-496.736338,-494.701394


## PRISON DATASET

In [9]:
LEVELS_prison = ['Australia', 'State', 'Gender', 'Legal', 'Indigenous']
data_prison = 'prison'
FC_TYPE = ['base', 'case1', 'case2', 'case3', 'case4', 'bottomup', 'ols', 'wls', 'mintshrink', 'erm']
column_order_1 = ['BU', 'OLS', 'WLS', 'MinT(Shrink)', 'ERM', 'ML_CASE1', 'ML_CASE2', 'ML_CASE3', 'ML_CASE4']

### ARIMA

In [10]:
model_arima ='arima'
base_errors_prison_arima, percentages_prison_arima = get_results(data_prison, model_arima, LEVELS_prison, FC_TYPE, column_order_1)

In [11]:
base_errors_prison_arima

error_metric,MAE,MSE,SMAPE
Australia,505.96,424138.76,1.33
State,369.95,385049.52,7.93
Gender,187.61,148874.19,11.22
Legal,111.09,48125.76,17.65
Indigenous,59.63,18835.82,21.90
Overall,114.37,71339.16,18.27


In [12]:
percentages_prison_arima['MAE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Shrink),ERM,ML_CASE1,ML_CASE2,ML_CASE3,ML_CASE4
Australia,-300.762906,-6.964977,-91.554668,-50.264843,-63.989248,-274.780615,-255.571587,-293.568662,-210.481066
State,8.365995,5.714286,13.599135,15.664279,0.583863,10.458170,8.149750,9.317475,15.210164
Gender,7.275732,5.212942,13.655988,15.516230,-3.048878,8.784180,6.119077,8.091253,12.696551
Legal,-0.531101,-0.684130,-1.395265,3.294626,-31.181925,0.720137,-0.387074,0.423080,4.311819
Indigenous,0.000000,0.301861,1.542848,4.662083,-30.420929,0.771424,-0.838504,0.553413,3.504947
Overall,-7.764274,2.011017,2.588091,7.012328,-19.270788,-5.508438,-6.601381,-6.723791,0.384716


In [13]:
percentages_prison_arima['MSE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Shrink),ERM,ML_CASE1,ML_CASE2,ML_CASE3,ML_CASE4
Australia,-1204.936229,-13.275846,-230.437789,-114.450766,-150.126572,-1075.443869,-995.093273,-1163.801370,-790.806183
State,-0.741349,8.906267,14.764202,20.707253,9.486221,2.844229,-3.302147,0.845512,13.523699
Gender,-4.889860,10.048874,13.384368,19.009192,15.612176,-0.642946,-6.228743,-2.903378,9.580364
Legal,-12.232326,-10.070241,-11.037498,-3.632026,-59.544452,-9.295874,-13.867729,-10.621962,-2.337709
Indigenous,0.000000,1.381039,1.571899,8.681385,-36.429845,1.838624,-0.855922,1.050764,6.404553
Overall,-63.001288,3.695151,-4.110239,7.575853,-15.393930,-53.406516,-54.385165,-59.431622,-30.909545


In [14]:
percentages_prison_arima['SMAPE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Shrink),ERM,ML_CASE1,ML_CASE2,ML_CASE3,ML_CASE4
Australia,-313.533835,-6.766917,-93.233083,-51.127820,-66.165414,-285.714286,-264.661654,-305.263158,-217.293233
State,4.918033,16.519546,9.836066,13.366961,-61.538462,6.305170,2.395965,4.539723,13.114754
Gender,0.178253,3.208556,2.762923,4.812834,-64.973262,1.336898,-7.486631,-2.852050,8.021390
Legal,-3.059490,18.583569,1.926346,2.549575,-46.968839,-4.079320,-5.665722,-6.345609,6.062323
Indigenous,0.000000,8.949772,3.561644,2.648402,-75.753425,-4.931507,-6.666667,-3.607306,3.242009
Overall,-0.821018,11.111111,3.174603,3.065134,-67.104543,-4.050356,-6.349206,-4.214559,4.488232


### ETS

In [15]:
model_ets ='ets'
base_errors_prison_ets, percentages_prison_ets = get_results(data_prison, model_ets, LEVELS_prison, FC_TYPE, column_order_1)

In [16]:
base_errors_prison_ets

error_metric,MAE,MSE,SMAPE
Australia,296.41,114530.91,0.78
State,385.07,418994.30,8.91
Gender,182.35,143906.22,12.69
Legal,107.81,51455.20,18.46
Indigenous,57.19,15931.03,21.72
Overall,110.78,69711.88,18.64


In [17]:
percentages_prison_ets['MAE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Shrink),ERM,ML_CASE1,ML_CASE2,ML_CASE3,ML_CASE4
Australia,-514.456327,-37.677541,-288.967984,-215.083837,-585.472825,-349.752033,-274.710705,-345.241389,-400.273270
State,19.804191,6.913029,16.612564,16.368452,15.823097,22.050536,22.585504,21.523359,22.385540
Gender,13.605703,-2.127776,10.408555,10.479846,-6.191390,14.867014,12.739238,14.888950,16.013162
Legal,-2.235414,0.176236,-3.598924,-1.094518,-16.733142,-2.003525,-0.742046,-1.864391,-0.667842
Indigenous,0.000000,-3.077461,-0.996678,0.681937,-37.961182,-0.419654,-0.052457,-0.419654,0.926735
Overall,-4.441235,-0.505506,-1.507492,1.191551,-25.329482,-0.063188,1.679003,-0.045135,-0.144430


In [18]:
percentages_prison_ets['MSE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Shrink),ERM,ML_CASE1,ML_CASE2,ML_CASE3,ML_CASE4
Australia,-3671.615741,-95.390013,-1439.641421,-932.717901,-5300.929077,-2305.810545,-1661.406165,-2263.394930,-2671.845793
State,23.569975,14.275249,20.715633,20.751485,-14.648431,27.477527,32.290862,27.765831,28.817375
Gender,12.276370,3.129698,7.906455,7.916996,-34.866193,17.715600,19.980999,17.820161,17.911554
Legal,6.081990,-0.466017,-4.214015,0.125546,-24.828064,6.718738,10.963110,6.729602,8.536397
Indigenous,0.000000,-5.287354,-14.794398,-7.243976,-86.333464,0.451634,3.355213,0.494130,2.080656
Overall,-35.948105,4.501743,-11.767894,-3.108093,-102.595354,-14.187037,-1.726851,-13.460762,-18.019368


In [19]:
percentages_prison_ets['SMAPE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Shrink),ERM,ML_CASE1,ML_CASE2,ML_CASE3,ML_CASE4
Australia,-530.769231,-38.461538,-294.871795,-217.948718,-605.128205,-356.410256,-278.205128,-351.282051,-410.256410
State,15.600449,28.619529,12.233446,15.151515,-37.485971,19.640853,16.947250,19.977553,19.304153
Gender,13.553980,-35.303388,10.165485,9.141056,-47.044917,14.578408,9.219858,18.203310,19.621749
Legal,2.112676,-5.146262,1.245937,4.062839,-70.368364,6.879740,-0.054171,9.696641,7.529794
Indigenous,0.000000,-16.620626,-1.243094,-0.920810,-98.296501,-1.519337,-9.622468,-0.138122,2.808471
Overall,2.092275,-13.894850,0.751073,1.716738,-84.656652,2.682403,-4.721030,4.613734,5.901288


## TOURISM DATASET

In [24]:
LEVELS_tourism = ['Australia', 'States', 'Regions']
data_tourism = 'tourism'
FC_TYPE = ['base', 'case1', 'case2', 'case3', 'case4', 'bottomup', 'ols', 'wls', 'mintsample', 'mintshrink', 'erm']
column_order_1 = ['BU', 'OLS', 'WLS', 'MinT(Shrink)', 'MinT(Sample)', 'ERM', 'ML_CASE1', 'ML_CASE2', 'ML_CASE3', 'ML_CASE4']

### ARIMA

In [25]:
model_arima ='arima'
base_errors_tourism_arima, percentages_tourism_arima = get_results(data_tourism, model_arima, LEVELS_tourism, FC_TYPE, column_order_1)

In [26]:
base_errors_tourism_arima

error_metric,MAE,MSE,SMAPE
Australia,2373.12,7695575.82,7.05
States,581.33,682870.53,16.33
Regions,101.65,29724.09,34.47
Overall,167.87,173699.11,32.66


In [27]:
percentages_tourism_arima['MAE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Shrink),MinT(Sample),ERM,ML_CASE1,ML_CASE2,ML_CASE3,ML_CASE4
Australia,-127.329423,-9.046740,-80.638147,-49.872320,-18.270041,9.184112,-11.950512,-16.291633,-48.307292,-93.304595
States,-36.896427,21.075809,-10.321160,5.289595,19.307450,24.082707,16.933583,17.208814,4.008051,-18.935888
Regions,0.000000,6.473192,7.752091,9.729464,11.293655,0.777177,6.935563,7.437285,6.089523,3.649779
Overall,-31.703104,8.053851,-12.104605,-1.453506,8.655507,8.822303,6.642044,6.278668,-3.550366,-18.919402


In [28]:
percentages_tourism_arima['MSE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Shrink),MinT(Sample),ERM,ML_CASE1,ML_CASE2,ML_CASE3,ML_CASE4
Australia,-336.612973,-15.402187,-186.242951,-102.650850,-32.443611,-0.952876,-21.482630,-31.344199,-93.752933,-222.854610
States,-69.790932,28.159025,-13.739921,12.530673,35.504637,42.812455,33.341014,32.881787,10.043589,-32.783373
Regions,0.000000,16.233769,17.287729,20.335997,23.978228,14.419214,17.139465,17.126916,13.681832,6.746111
Overall,-198.046179,3.605229,-98.842832,-46.294762,-1.698437,15.599435,2.254065,-3.036648,-43.493683,-125.725319


In [29]:
percentages_tourism_arima['SMAPE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Shrink),MinT(Sample),ERM,ML_CASE1,ML_CASE2,ML_CASE3,ML_CASE4
Australia,-139.574468,-9.361702,-86.382979,-52.482270,-18.723404,10.212766,-11.489362,-15.460993,-51.631206,-101.134752
States,-10.165340,27.005511,5.756277,19.840784,30.128598,25.780772,22.351500,23.943662,17.085119,3.306797
Regions,0.000000,2.727009,3.713374,6.788512,7.484769,-17.493473,3.394256,4.612707,4.786771,3.771395
Overall,-0.765462,3.704838,3.582364,7.195346,8.389467,-15.615432,4.164115,5.358236,5.174525,3.521127


### ETS

In [30]:
model_ets ='ets'
base_errors_tourism_ets, percentages_tourism_ets = get_results(data_tourism, model_ets, LEVELS_tourism, FC_TYPE, column_order_1)

In [31]:
base_errors_tourism_ets

error_metric,MAE,MSE,SMAPE
Australia,3149.11,12915938.49,9.62
States,569.86,610351.60,13.32
Regions,91.50,23802.54,32.26
Overall,166.87,223778.77,30.44


In [32]:
percentages_tourism_ets['MAE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Shrink),MinT(Sample),ERM,ML_CASE1,ML_CASE2,ML_CASE3,ML_CASE4
Australia,-37.796393,1.198434,-15.479929,-9.772920,19.036490,25.515146,20.777617,-26.729774,-7.201400,-20.576607
States,-19.446882,-1.533710,-5.153897,-1.923279,20.575229,-15.500298,21.596532,-12.245113,-0.619450,-8.730214
Regions,0.000000,1.770492,4.295082,4.218579,1.551913,-60.524590,5.639344,1.431694,3.071038,1.857923
Overall,-13.861089,0.719123,-2.750644,-0.617247,10.786840,-28.758914,13.489543,-8.665428,-0.245700,-6.100557


In [33]:
percentages_tourism_ets['MSE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Shrink),MinT(Sample),ERM,ML_CASE1,ML_CASE2,ML_CASE3,ML_CASE4
Australia,-74.350078,0.996985,-32.792912,-23.578552,26.454736,34.256214,29.719856,-49.074116,-16.094326,-39.891831
States,-37.322787,2.967490,-8.087188,-2.283716,31.235177,-36.878294,31.900888,-23.789988,-4.277852,-18.712472
Regions,0.000000,6.104559,10.261384,10.001538,7.754256,-111.025210,12.071653,2.311098,5.514706,3.083957
Overall,-58.869128,1.931734,-23.095100,-15.559769,25.726605,4.279624,28.509250,-38.443656,-11.358008,-30.993659


In [34]:
percentages_tourism_ets['SMAPE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Shrink),MinT(Sample),ERM,ML_CASE1,ML_CASE2,ML_CASE3,ML_CASE4
Australia,-41.683992,0.519751,-18.399168,-12.474012,18.711019,28.898129,20.478170,-29.625780,-8.419958,-22.972973
States,-16.066066,-21.621622,-4.954955,-1.426426,11.636637,-42.192192,16.366366,-6.756757,1.576577,-3.453453
Regions,0.000000,-2.944823,1.890887,2.138872,-1.239926,-71.109733,4.153751,2.572846,3.936764,3.130812
Overall,-0.722733,-3.580815,1.609724,1.971091,-0.689882,-69.645204,4.697766,2.135348,3.843627,2.825230


### Forecasts across seeds

In [35]:
data = 'prison'
model = 'ets'
path = '../results/validation_results_experiments/'

fc_seeds = pd.read_csv(f'{path}{data}_{model}_seed_runs.csv', index_col =0)

In [36]:
std_results = fc_seeds.groupby(fc_seeds.index).std()

In [37]:
std_results

,0,1,2,3,4,5,6,7
ACT-Female-Remanded-ATSI,2.369480,2.413365,2.372096,2.400927,2.586641,2.634543,2.596754,2.627631
ACT-Female-Remanded-Non-ATSI,1.606393,1.611455,1.552460,1.553457,1.686703,1.692010,1.636631,1.638402
ACT-Female-Sentenced-ATSI,1.728251,1.762741,1.751605,1.765333,1.883511,1.920292,1.908104,1.922641
ACT-Female-Sentenced-Non-ATSI,1.383259,1.423514,1.415436,1.432201,1.494423,1.537753,1.530201,1.548333
ACT-Male-Remanded-ATSI,1.468645,1.516005,1.482083,1.491256,1.631360,1.687992,1.650392,1.662732
...,...,...,...,...,...,...,...,...
WA-Female-Sentenced-Non-ATSI,2.197882,2.236175,2.146643,2.155461,2.397064,2.437066,2.350999,2.360602
WA-Male-Remanded-ATSI,1.978030,2.007106,1.993296,2.020706,2.194336,2.230870,2.220372,2.251134
WA-Male-Remanded-Non-ATSI,3.387002,3.428934,3.323050,3.334754,3.617367,3.655736,3.549880,3.563819
WA-Male-Sentenced-ATSI,12.342096,12.403200,11.969615,11.982837,13.104752,13.161222,12.741746,12.766034


In [34]:
std_results.to_csv(f'{path}{data}_{model}_std_results.csv')

### Comparing with previous parameters

In [39]:
LEVELS_tourism = ['Australia', 'States', 'Regions']
data_tourism = 'tourism'
model_ets ='ets'

FC_TYPE = ['base', 'case1', 'bottomup', 'ols', 'wls', 'mintsample', 'mintshrink', 'erm', 'case1_mean', 'case1_one_seed',
          'seeds_prev_params', 'one_seed_prev_params', 'seeds_prev_params_mean']
column_order_1 = ['BU', 'OLS', 'WLS', 'MinT(Sample)', 'MinT(Shrink)', 'ERM', 
                  'ML_CASE1', 'ML_CASE1_MEAN', 'ML_CASE1_ONE_SEED', 'PREV_PARAMS_SEEDS',
                 'PREV_PARAMS_ONE_SEED', 'PREV_PARAMS_SEEDS_MEAN']


base_errors_tourism_ets, percentages_tourism_ets = get_results(data_tourism, model_ets, 
                                                                           LEVELS_tourism, FC_TYPE, column_order_1)

In [40]:
percentages_tourism_ets['MAE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Sample),MinT(Shrink),ERM,ML_CASE1,ML_CASE1_MEAN,ML_CASE1_ONE_SEED,PREV_PARAMS_SEEDS,PREV_PARAMS_ONE_SEED,PREV_PARAMS_SEEDS_MEAN
Australia,-37.796393,1.198434,-15.479929,19.036490,-9.772920,25.515146,20.777617,30.461940,0.606521,6.924179,-20.307642,2.260004
States,-19.446882,-1.533710,-5.153897,20.575229,-1.923279,-15.500298,21.596532,26.171340,6.696382,12.434633,-7.484294,8.314323
Regions,0.000000,1.770492,4.295082,1.551913,4.218579,-60.524590,5.639344,5.464481,4.426230,5.103825,2.437158,4.535519
Overall,-13.861089,0.719123,-2.750644,10.786840,-0.617247,-28.758914,13.489543,16.839456,4.218853,7.568766,-5.399413,5.093786


In [41]:
percentages_tourism_ets['MSE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Sample),MinT(Shrink),ERM,ML_CASE1,ML_CASE1_MEAN,ML_CASE1_ONE_SEED,PREV_PARAMS_SEEDS,PREV_PARAMS_ONE_SEED,PREV_PARAMS_SEEDS_MEAN
Australia,-74.350078,0.996985,-32.792912,26.454736,-23.578552,34.256214,29.719856,43.262761,0.189414,11.730764,-36.344570,3.509987
States,-37.322787,2.967490,-8.087188,31.235177,-2.283716,-36.878294,31.900888,39.569137,10.096322,19.785266,-13.205107,12.962755
Regions,0.000000,6.104559,10.261384,7.754256,10.001538,-111.025210,12.071653,13.491963,8.331422,9.636451,4.192242,8.543542
Overall,-58.869128,1.931734,-23.095100,25.726605,-15.559769,4.279624,28.509250,39.564535,3.199186,13.338137,-27.241136,6.118239


ARIMA

In [43]:
LEVELS_tourism = ['Australia', 'States', 'Regions']
data_tourism = 'tourism'
model_arima ='arima'



base_errors_tourism_arima, percentages_tourism_arima = get_results(data_tourism, model_arima, 
                                                                                     LEVELS_tourism, FC_TYPE, column_order_1)

In [44]:
percentages_tourism_arima['MAE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Sample),MinT(Shrink),ERM,ML_CASE1,ML_CASE1_MEAN,ML_CASE1_ONE_SEED,PREV_PARAMS_SEEDS,PREV_PARAMS_ONE_SEED,PREV_PARAMS_SEEDS_MEAN
Australia,-127.329423,-9.046740,-80.638147,-18.270041,-49.872320,9.184112,-11.950512,-8.662857,-28.028924,-39.036795,30.475071,-32.677235
States,-36.896427,21.075809,-10.321160,19.307450,5.289595,24.082707,16.933583,17.757556,11.138252,7.947293,35.767980,10.479418
Regions,0.000000,6.473192,7.752091,11.293655,9.729464,0.777177,6.935563,6.915888,6.837186,6.109198,5.882932,6.492868
Overall,-31.703104,8.053851,-12.104605,8.655507,-1.453506,8.822303,6.642044,7.410496,2.263656,-0.875678,18.496456,1.113957


In [45]:
percentages_tourism_arima['MSE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Sample),MinT(Shrink),ERM,ML_CASE1,ML_CASE1_MEAN,ML_CASE1_ONE_SEED,PREV_PARAMS_SEEDS,PREV_PARAMS_ONE_SEED,PREV_PARAMS_SEEDS_MEAN
Australia,-336.612973,-15.402187,-186.242951,-32.443611,-102.650850,-0.952876,-21.482630,-16.108026,-52.150190,-72.691566,51.758699,-61.281425
States,-69.790932,28.159025,-13.739921,35.504637,12.530673,42.812455,33.341014,34.873838,23.396958,16.367136,59.216874,20.000680
Regions,0.000000,16.233769,17.287729,23.978228,20.335997,14.419214,17.139465,17.269965,15.305229,14.355696,21.386021,14.998138
Overall,-198.046179,3.605229,-98.842832,-1.698437,-46.294762,15.599435,2.254065,5.571934,-17.234412,-30.364232,49.465020,-23.141005


In [46]:
new_df = percentages_tourism_arima['MSE'].iloc[:, 0:8]
new_df.style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Sample),MinT(Shrink),ERM,ML_CASE1,ML_CASE1_MEAN
Australia,-336.612973,-15.402187,-186.242951,-32.443611,-102.650850,-0.952876,-21.482630,-16.108026
States,-69.790932,28.159025,-13.739921,35.504637,12.530673,42.812455,33.341014,34.873838
Regions,0.000000,16.233769,17.287729,23.978228,20.335997,14.419214,17.139465,17.269965
Overall,-198.046179,3.605229,-98.842832,-1.698437,-46.294762,15.599435,2.254065,5.571934


### Prison

In [48]:
LEVELS_prison = ['Australia', 'State', 'Gender', 'Legal', 'Indigenous']
data_prison = 'prison'
model_ets ='ets'


FC_TYPE = ['base', 'case1', 'bottomup', 'ols', 'wls', 'mintshrink', 'erm', 'case1_mean', 'case1_one_seed',
          'seeds_prev_params', 'one_seed_prev_params', 'seeds_prev_params_mean']
column_order_1 = ['BU', 'OLS', 'WLS', 'MinT(Shrink)', 'ERM', 
                  'ML_CASE1', 'ML_CASE1_MEAN', 'ML_CASE1_ONE_SEED', 'PREV_PARAMS_SEEDS',
                 'PREV_PARAMS_ONE_SEED', 'PREV_PARAMS_SEEDS_MEAN']

base_errors_prison_ets, percentages_prison_ets = get_results(data_prison, model_ets, 
                                                                                LEVELS_prison, FC_TYPE,
                                                                               column_order_1)

In [49]:
percentages_prison_ets['MSE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Shrink),ERM,ML_CASE1,ML_CASE1_MEAN,ML_CASE1_ONE_SEED,PREV_PARAMS_SEEDS,PREV_PARAMS_ONE_SEED,PREV_PARAMS_SEEDS_MEAN
Australia,-3671.615741,-95.390013,-1439.641421,-932.717901,-5300.929077,-2305.810545,-2452.088480,-2120.722738,-3004.996311,-3122.802176,-2932.347547
State,23.569975,14.275249,20.715633,20.751485,-14.648431,27.477527,23.676155,30.502919,27.649837,26.801964,28.592761
Gender,12.276370,3.129698,7.906455,7.916996,-34.866193,17.715600,12.686053,20.291798,16.670384,15.576519,17.634894
Legal,6.081990,-0.466017,-4.214015,0.125546,-24.828064,6.718738,3.926911,7.966620,8.273333,7.526936,8.745161
Indigenous,0.000000,-5.287354,-14.794398,-7.243976,-86.333464,0.451634,-1.790280,1.448180,1.770821,1.152593,2.174185
Overall,-35.948105,4.501743,-11.767894,-3.108093,-102.595354,-14.187037,-19.872624,-9.404466,-23.434384,-25.889877,-21.669133


In [50]:
new_df_prison = percentages_prison_ets['MSE'].iloc[:, [0,1,2,3,4,6]]
new_df_prison.style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Shrink),ERM,ML_CASE1_MEAN
Australia,-3671.615741,-95.390013,-1439.641421,-932.717901,-5300.929077,-2452.088480
State,23.569975,14.275249,20.715633,20.751485,-14.648431,23.676155
Gender,12.276370,3.129698,7.906455,7.916996,-34.866193,12.686053
Legal,6.081990,-0.466017,-4.214015,0.125546,-24.828064,3.926911
Indigenous,0.000000,-5.287354,-14.794398,-7.243976,-86.333464,-1.790280
Overall,-35.948105,4.501743,-11.767894,-3.108093,-102.595354,-19.872624


ARIMA

In [51]:
model_arima ='arima'
base_errors_prison_arima, percentages_prison_arima = get_results(data_prison, model_arima, 
                                                                                LEVELS_prison, FC_TYPE,
                                                                               column_order_1)

In [52]:
percentages_prison_arima['MSE'].style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Shrink),ERM,ML_CASE1,ML_CASE1_MEAN,ML_CASE1_ONE_SEED,PREV_PARAMS_SEEDS,PREV_PARAMS_ONE_SEED,PREV_PARAMS_SEEDS_MEAN
Australia,-1204.936229,-13.275846,-230.437789,-114.450766,-150.126572,-1075.443869,-1059.088328,-1215.842289,-1032.473755,-701.349349,-994.539608
State,-0.741349,8.906267,14.764202,20.707253,9.486221,2.844229,3.681155,-3.048964,5.425055,17.297824,6.966218
Gender,-4.889860,10.048874,13.384368,19.009192,15.612176,-0.642946,0.294866,-6.856911,1.642508,11.729790,2.810964
Legal,-12.232326,-10.070241,-11.037498,-3.632026,-59.544452,-9.295874,-8.703468,-13.561116,-7.948945,-0.034846,-6.767955
Indigenous,0.000000,1.381039,1.571899,8.681385,-36.429845,1.838624,2.290954,-0.960457,2.601957,7.930210,3.481717
Overall,-63.001288,3.695151,-4.110239,7.575853,-15.393930,-53.406516,-51.876571,-65.274640,-49.396601,-23.950184,-46.326730


In [54]:
new_df_prison = percentages_prison_arima['MSE'].iloc[:, 0:8]
new_df_prison.style.highlight_max(color = 'lightgreen', axis = 1)

,BU,OLS,WLS,MinT(Shrink),ERM,ML_CASE1,ML_CASE1_MEAN,ML_CASE1_ONE_SEED
Australia,-1204.936229,-13.275846,-230.437789,-114.450766,-150.126572,-1075.443869,-1059.088328,-1215.842289
State,-0.741349,8.906267,14.764202,20.707253,9.486221,2.844229,3.681155,-3.048964
Gender,-4.889860,10.048874,13.384368,19.009192,15.612176,-0.642946,0.294866,-6.856911
Legal,-12.232326,-10.070241,-11.037498,-3.632026,-59.544452,-9.295874,-8.703468,-13.561116
Indigenous,0.000000,1.381039,1.571899,8.681385,-36.429845,1.838624,2.290954,-0.960457
Overall,-63.001288,3.695151,-4.110239,7.575853,-15.393930,-53.406516,-51.876571,-65.274640
